In [1]:
! pip install redis

  Using cached redis-3.5.3-py2.py3-none-any.whl (72 kB)


In [2]:
import redis
r = redis.Redis(host="my_redis")
print(r.ping())

True


In [3]:
r.set('temperature', 18.5)

True

In [4]:
r.get('temperature')

b'18.5'

In [72]:
import time
r.flushall() # delete ALL keys in the DB
TTL = 5
r.set('temperature', 18.5)
r.expire('temperature', TTL)  # the key will be deleted after TTL seconds
print("Temperature =", r.get('temperature'))
print("now sleeping for %s seconds..." % (TTL+1))
time.sleep(TTL+1)
print("Temperature =", r.get('temperature'))

Temperature = b'18.5'
now sleeping for 6 seconds...
Temperature = None


In [8]:
import time
r.flushall()  # delete ALL keys in the DB

start = time.time()
N = 20000
for i in range(N):
    key = "key%s" % i
    value = "value%s" % i
    r.set(key, value)
delta = time.time() - start

print("set: %d items in %.02f seconds"% (N, delta), end=' ')
print("(%.02f items/sec)" % (N/delta))

start = time.time()
N = 20000
for i in range(N):
    key = "key%s" % i
    value = r.get(key)
delta = time.time() - start

print("get: %d items in %.02f seconds"% (N, delta), end=' ')
print("(%.02f items/sec)" % (N/delta))

set: 20000 items in 9.40 seconds (2127.85 items/sec)
get: 20000 items in 8.82 seconds (2267.18 items/sec)


In [11]:
import time
r.flushall()  # delete ALL keys in the DB

start = time.time()
pipe = r.pipeline()
N = 20000
for i in range(N):
    key = "key%s" % i
    value = "value%s" % i
    pipe.set(key, value)
pipe.execute()
delta = time.time() - start

print("set: %d items in %.02f seconds"% (N, delta), end=' ')
print("(%.02f items/sec)" % (N/delta))

start = time.time()
pipe = r.pipeline()
N = 20000
for i in range(N):
    key = "key%s" % i
    value = pipe.get(key)
pipe.execute()
delta = time.time() - start

print("get: %d items in %.02f seconds"% (N, delta), end=' ')
print("(%.02f items/sec)" % (N/delta))

set: 20000 items in 0.91 seconds (22076.32 items/sec)
get: 20000 items in 0.68 seconds (29526.88 items/sec)


In [67]:
import sqlite3 as sql
conn = sql.connect('test_perf.sqlite')
cur = conn.cursor()
cur.execute('''DROP TABLE IF EXISTS Performance;''')
cur.execute('''CREATE TABLE Performance(
                key TEXT NOT NULL UNIQUE,
                value TEXT NOT NULL);
            ''')
conn.commit()
conn.close()

start = time.time()
conn = sql.connect('test_perf.sqlite')
cur = conn.cursor()
N = 20000
for i in range(N):
    key = "key%s" % i
    value = "value%s" % i
    cur.execute('''INSERT INTO Performance VALUES(?, ?)''', (key,value))
conn.commit()
conn.close()
delta = time.time() - start

print("SQLite set: %d items in %.02f seconds"% (N, delta), end=' ')
print("(%.02f items/sec)" % (N/delta))

start = time.time()
conn = sql.connect('test_perf.sqlite')
cur = conn.cursor()
cur.execute('''SELECT * from Performance''')
results = cur.fetchall()
for res in results:
    (key, value) = res
conn.close()
delta = time.time() - start

print("SQLite get: %d items in %.02f seconds"% (N, delta), end=' ')
print("(%.02f items/sec)" % (N/delta))

SQLite set: 20000 items in 0.42 seconds (47515.79 items/sec)
SQLite get: 20000 items in 0.20 seconds (102349.03 items/sec)


In [60]:
import time
r.flushall()  # delete ALL keys in the DB

start = time.time()
N = 20000
my_dict = {"key%s" % i: "value%s" % i for i in range(N)}
r.mset(my_dict)
delta = time.time() - start

print("mset: %d items in %.02f seconds"% (N, delta), end=' ')
print("(%.02f items/sec)" % (N/delta))

start = time.time()
N = 20000
keys = ["key%s" % i for i in range(N)]
values = r.mget(keys)
results = list(zip(keys, values))
delta = time.time() - start

print("mget: %d items in %.02f seconds"% (N, delta), end=' ')
print("(%.02f items/sec)" % (N/delta))

mset: 20000 items in 0.23 seconds (87975.07 items/sec)
mget: 20000 items in 0.16 seconds (129017.99 items/sec)


In [82]:
# a subscriber subscribes to the "bdb" channel...
a_subscriber = redis.Redis(host="my_redis")
sub = a_subscriber.pubsub()
sub.subscribe('bdb')

# ... and then gets messages over that channel
print("First get: ", sub.get_message())
print("Second get: ", sub.get_message())

First get:  {'type': 'subscribe', 'pattern': None, 'channel': b'bdb', 'data': 1}
Second get:  None


In [83]:
# a publisher publishes something on the "bdb" channel
a_publisher = redis.Redis(host="my_redis")
a_publisher.publish('bdb', 'pubsub test')

# the subscriber gets another messsage... this time it can read it
print("Third get: ", sub.get_message())

Third get:  {'type': 'message', 'pattern': None, 'channel': b'bdb', 'data': b'pubsub test'}
